In [1]:
import yfinance as yf
import datetime
import pandas as pd

In [6]:
symbols_list = ["RELIANCE.NS"]#, "TCS.NS", "INFY.NS"]
df = yf.download(symbols_list, period="1d", interval="15m")
# df = df.T

print(df.head(20))

[*********************100%***********************]  1 of 1 completed

Price                            Close         High          Low         Open  \
Ticker                     RELIANCE.NS  RELIANCE.NS  RELIANCE.NS  RELIANCE.NS   
Datetime                                                                        
2025-12-29 03:45:00+00:00  1554.099976  1556.900024  1551.500000  1555.199951   
2025-12-29 04:00:00+00:00  1557.300049  1557.800049  1552.300049  1554.000000   
2025-12-29 04:15:00+00:00  1553.699951  1557.699951  1553.599976  1557.400024   
2025-12-29 04:30:00+00:00  1551.400024  1554.400024  1551.400024  1553.199951   
2025-12-29 04:45:00+00:00  1551.599976  1552.800049  1549.800049  1551.500000   
2025-12-29 05:00:00+00:00  1548.000000  1552.000000  1547.800049  1551.599976   
2025-12-29 05:15:00+00:00  1549.400024  1550.300049  1547.400024  1548.000000   
2025-12-29 05:30:00+00:00  1549.400024  1549.900024  1547.699951  1549.300049   
2025-12-29 05:45:00+00:00  1547.599976  1549.900024  1546.500000  1549.500000   
2025-12-29 06:00:00+00:00  1

In [18]:
def yt_finance_historical_data(symbols_list):
    end = datetime.date.today()
    start = end - datetime.timedelta(days=40)  # enough buffer for 30 trading days
    df = yf.download(symbols_list, start=start, end=end, interval="1d")
    last_6 = df.tail(6)["Close"]
    last_30 = df.tail(40)["Close"]

    last_6, last_30 = last_6.T, last_30.T
    last_6 = last_6.reset_index()
    last_30 = last_30.reset_index()

    last_6['SYMBOL'] = last_6['Ticker'].str.replace('.NS', '')
    last_30['SYMBOL'] = last_30['Ticker'].str.replace('.NS', '')

    # Fill NaN values in date columns starting from the second date column
    date_cols_6 = last_6.columns[1:-1]  # exclude 'Ticker' and 'SYMBOL'
    date_cols_30 = last_30.columns[1:-1]  # exclude 'Ticker' and 'SYMBOL'
    last_6[date_cols_6[1:]] = last_6[date_cols_6[1:]].fillna(method='ffill', axis=1)
    last_30[date_cols_30[1:]] = last_30[date_cols_30[1:]].fillna(method='ffill', axis=1)

    # For last_6
    last_date_6 = date_cols_6[-1]
    fourth_from_last_6 = date_cols_6[-4]
    last_6['DIFF'] = last_6[last_date_6] - last_6[fourth_from_last_6]
    last_6['Diff_percent'] = last_6['DIFF'] *100 / last_6[fourth_from_last_6]

    # For last_30
    last_date_30 = date_cols_30[-1]
    fourth_from_last_30 = date_cols_30[-4]
    last_30['DIFF'] = last_30[last_date_30] - last_30[fourth_from_last_30]
    last_30['Diff_percent'] = last_30['DIFF'] *100 / last_30[fourth_from_last_30]

    # Sort by Diff_percent in descending order
    last_6 = last_6.sort_values(by='Diff_percent', ascending=False)
    last_30 = last_30.sort_values(by='Diff_percent', ascending=False)

    return last_6, last_30

In [19]:
# print(last_6.head(20))
# print(last_30.head(40))

In [20]:
df_metadata = pd.read_csv("instruments_data.csv")
df_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 523 entries, 0 to 522
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Slno        523 non-null    int64  
 1   Instrument  523 non-null    object 
 2   Type        306 non-null    object 
 3   No          523 non-null    object 
 4   F           523 non-null    int64  
 5   1218        523 non-null    int64  
 6   1219        523 non-null    int64  
 7   1222        523 non-null    int64  
 8   1223        523 non-null    int64  
 9   1224        523 non-null    int64  
 10  1226        523 non-null    int64  
 11  Dif         523 non-null    int64  
 12  Percent     523 non-null    float64
 13  W           523 non-null    int64  
dtypes: float64(1), int64(10), object(3)
memory usage: 57.3+ KB


In [21]:
df_metadata.rename(columns={'Type': 'cap_category', 'W': 'symbol_category', 'F': 'personal_rating'}, inplace=True)
df_metadata = df_metadata[['Instrument', 'cap_category', 'personal_rating', 'symbol_category']]

In [22]:
df_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 523 entries, 0 to 522
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Instrument       523 non-null    object
 1   cap_category     306 non-null    object
 2   personal_rating  523 non-null    int64 
 3   symbol_category  523 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 16.5+ KB


In [23]:
df_metadata.drop_duplicates(inplace=True)

In [24]:
df_metadata.info()

<class 'pandas.core.frame.DataFrame'>
Index: 520 entries, 0 to 522
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Instrument       520 non-null    object
 1   cap_category     305 non-null    object
 2   personal_rating  520 non-null    int64 
 3   symbol_category  520 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 20.3+ KB


In [25]:
temp = list(df_metadata['Instrument'])
print(len(temp))
temp = list(set(temp))
print(len(temp))

520
511


In [26]:
df_metadata.sort_values(by='Instrument', inplace=True)

In [27]:
df_metadata.head()

,Instrument,cap_category,personal_rating,symbol_category
124,360ONE,L,1,2
288,AADHARHFC,L,0,1
287,AADHARHFC,NaN,0,1
191,AARTIIND,NaN,0,0
332,AARTIND,NaN,0,0


In [28]:
df_metadata.reset_index(drop=True, inplace=True)

In [29]:
df_metadata.head()

,Instrument,cap_category,personal_rating,symbol_category
0,360ONE,L,1,2
1,AADHARHFC,L,0,1
2,AADHARHFC,NaN,0,1
3,AARTIIND,NaN,0,0
4,AARTIND,NaN,0,0


In [40]:
df_metadata.to_csv("METADATA.csv", index=False)

In [ ]:
sample_list = df_metadata["Instrument"].to_list()
sample_list = [sample+".NS" for sample in sample_list]

In [31]:
last_6, last_30 = yt_finance_historical_data(sample_list)

[**                     4%                       ]  19 of 511 completed$GANESHHOUC.NS: possibly delisted; no timezone found
[**                     4%                       ]  20 of 511 completed$GANESHHOUC.NS: possibly delisted; no timezone found
[****                   8%                       ]  40 of 511 completed$ASAHIINDIAA.NS: possibly delisted; no timezone found
[****                   8%                       ]  41 of 511 completed$ASAHIINDIAA.NS: possibly delisted; no timezone found
[**********************50%                       ]  255 of 511 completed$TATAMOTORS.NS: possibly delisted; no timezone found
[**********************50%                       ]  256 of 511 completed$TATAMOTORS.NS: possibly delisted; no timezone found
[**********************55%*                      ]  282 of 511 completed$DRAGARWQ.NS: possibly delisted; no timezone found
[**********************56%**                     ]  284 of 511 completed$DRAGARWQ.NS: possibly delisted; no timezone found
[*****

In [32]:
last_6.info()

<class 'pandas.core.frame.DataFrame'>
Index: 511 entries, 245 to 449
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Ticker               511 non-null    object 
 1   2025-12-18 00:00:00  499 non-null    float64
 2   2025-12-19 00:00:00  497 non-null    float64
 3   2025-12-22 00:00:00  497 non-null    float64
 4   2025-12-23 00:00:00  499 non-null    float64
 5   2025-12-24 00:00:00  499 non-null    float64
 6   2025-12-25 00:00:00  499 non-null    float64
 7   SYMBOL               511 non-null    object 
 8   DIFF                 497 non-null    float64
 9   Diff_percent         497 non-null    float64
dtypes: float64(8), object(2)
memory usage: 43.9+ KB


In [34]:
last_6.head(10)

Date,Ticker,2025-12-18 00:00:00,2025-12-19 00:00:00,2025-12-22 00:00:00,2025-12-23 00:00:00,2025-12-24 00:00:00,2025-12-25 00:00:00,SYMBOL,DIFF,Diff_percent
245,JBMA.NS,543.900024,572.099976,572.450012,577.950012,639.950012,639.950012,JBMA,67.500000,11.791423
261,JWL.NS,253.949997,260.250000,310.450012,335.899994,340.600006,340.600006,JWL,30.149994,9.711706
235,IRCON.NS,150.009995,152.949997,156.809998,169.300003,170.699997,170.699997,IRCON,13.889999,8.857853
239,ISGEC.NS,804.200012,814.650024,833.250000,848.900024,906.750000,906.750000,ISGEC,73.500000,8.820882
412,SILVERBEES.NS,193.259995,192.559998,192.559998,201.789993,209.229996,209.229996,SILVERBEES,16.669998,8.657041
272,KNRCON.NS,142.100006,146.460007,157.500000,159.729996,171.020004,171.020004,KNRCON,13.520004,8.584130
501,WEBELSOLAR.NS,82.940002,84.070000,86.690002,85.489998,93.930000,93.930000,WEBELSOLAR,7.239998,8.351595
201,HINDCOPPER.NS,387.450012,387.549988,403.399994,407.799988,436.549988,436.549988,HINDCOPPER,33.149994,8.217649
385,RPOWER.NS,37.779999,38.560001,35.279999,36.279999,38.160000,38.160000,RPOWER,2.880001,8.163269
218,IFCI.NS,46.810001,47.919998,49.410000,52.970001,53.070000,53.070000,IFCI,3.660000,7.407407


In [38]:
merged_df = pd.merge(df_metadata, last_6, left_on='Instrument', right_on='SYMBOL', how="left")
merged_df = merged_df.sort_values(by='Diff_percent', ascending=False)

In [39]:

merged_df.head()

,Instrument,cap_category,personal_rating,symbol_category,Ticker,2025-12-18 00:00:00,2025-12-19 00:00:00,2025-12-22 00:00:00,2025-12-23 00:00:00,2025-12-24 00:00:00,2025-12-25 00:00:00,SYMBOL,DIFF,Diff_percent
247,JBMA,NaN,0,2,JBMA.NS,543.900024,572.099976,572.450012,577.950012,639.950012,639.950012,JBMA,67.500000,11.791423
264,JWL,M,0,2,JWL.NS,253.949997,260.250000,310.450012,335.899994,340.600006,340.600006,JWL,30.149994,9.711706
236,IRCON,M,0,4,IRCON.NS,150.009995,152.949997,156.809998,169.300003,170.699997,170.699997,IRCON,13.889999,8.857853
241,ISGEC,M,0,5,ISGEC.NS,804.200012,814.650024,833.250000,848.900024,906.750000,906.750000,ISGEC,73.500000,8.820882
420,SILVERBEES,NaN,0,4,SILVERBEES.NS,193.259995,192.559998,192.559998,201.789993,209.229996,209.229996,SILVERBEES,16.669998,8.657041
